# NDVI Prophet Model Training

This notebook helps to train a Facebook Prophet model using cached NDVI and weather data from the HLS Crop Monitor project to predict vegetation patterns for the current year.

## Objectives:
- Load cached NDVI and weather data from multiple years
- Train a Prophet model with weather variables as additional regressors  
- Generate predictions for the current year
- Evaluate model performance and visualize results

In [22]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import hashlib
from pathlib import Path
import warnings
import pickle
import logging

# Prophet and related libraries
try:
    from prophet import Prophet
    from prophet.diagnostics import cross_validation, performance_metrics
    from prophet.plot import plot_cross_validation_metric
except ImportError:
    print("Prophet not installed. Please install it using: pip install prophet")

# Plotly for interactive visualizations
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Weather data
import requests

# Set up logging and warnings
warnings.filterwarnings("ignore")
logging.getLogger('prophet').setLevel(logging.WARNING)

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


## Configuration and Cache Directory Setup

In [23]:
# Configuration from chipnik_monitor.py
CACHE_ROOT = Path(".cache")
STAC_CACHE_DIR = CACHE_ROOT / "stac"
NDVI_CACHE_DIR = CACHE_ROOT / "NDVI"
EVI_CACHE_DIR = CACHE_ROOT / "EVI"

# Model output directory
MODEL_OUTPUT_DIR = Path("models")
MODEL_OUTPUT_DIR.mkdir(exist_ok=True)

# Current year for predictions
CURRENT_YEAR = datetime.now().year

# Area of Interest (example - can be modified)
DEFAULT_BBOX = [-112.8480 - 0.1, 42.7730 - 0.1, -112.8480 + 0.1, 42.7730 + 0.1]  # Snake River Plain

print(f"📁 Cache directories:")
print(f"   STAC cache: {STAC_CACHE_DIR}")
print(f"   NDVI cache: {NDVI_CACHE_DIR}")
print(f"   EVI cache: {EVI_CACHE_DIR}")
print(f"🤖 Model output: {MODEL_OUTPUT_DIR}")
print(f"📅 Current year: {CURRENT_YEAR}")

📁 Cache directories:
   STAC cache: .cache\stac
   NDVI cache: .cache\NDVI
   EVI cache: .cache\EVI
🤖 Model output: models
📅 Current year: 2025


In [24]:
from chipnik_monitor import load_cached_data_with_region, list_stac_cache_files, fetch_weather_history

def calculate_vpd(temperature_c, relative_humidity_pct):
    """
    Calculate Vapor Pressure Deficit (VPD) from temperature and humidity.
    
    Parameters:
    temperature_c (float): Temperature in Celsius
    relative_humidity_pct (float): Relative humidity percentage (0-100)
    
    Returns:
    float: VPD in kPa
    """
    # Saturated vapor pressure (kPa) using Tetens equation
    es = 0.6108 * np.exp(17.27 * temperature_c / (temperature_c + 237.3))
    
    # Actual vapor pressure (kPa)
    ea = es * (relative_humidity_pct / 100.0)
    
    # VPD is the difference
    vpd = es - ea
    
    return np.maximum(vpd, 0)  # Ensure non-negative

def fetch_weather_data_for_period(lat, lon, start_date, end_date):
    """
    Wrapper function to fetch weather data with additional derived variables.
    
    Parameters:
    lat (float): Latitude
    lon (float): Longitude  
    start_date (datetime.date): Start date
    end_date (datetime.date): End date
    
    Returns:
    pd.DataFrame: Weather data with derived variables
    """
    # Convert dates to datetime objects if they're date objects
    if hasattr(start_date, 'strftime') and not hasattr(start_date, 'hour'):
        start_date = datetime.combine(start_date, datetime.min.time())
    if hasattr(end_date, 'strftime') and not hasattr(end_date, 'hour'):
        end_date = datetime.combine(end_date, datetime.min.time())
    
    # Fetch base weather data
    weather_df = fetch_weather_history(lat, lon, start_date, end_date)
    
    if weather_df.empty:
        return weather_df
    
    # Add derived variables expected by the Prophet model
    if 'temperature_mean' in weather_df.columns and 'humidity_mean' in weather_df.columns:
        # Rename humidity column to match expected name
        weather_df = weather_df.rename(columns={'humidity_mean': 'humidity'})
        
        # Calculate VPD
        weather_df['vapor_pressure_deficit'] = calculate_vpd(
            weather_df['temperature_mean'], 
            weather_df['humidity']
        )
        
        # Calculate growing degree days (base temperature 10°C)
        weather_df['growing_degree_days'] = np.maximum(
            weather_df['temperature_mean'] - 10, 0
        )
        
        # Add precipitation (synthetic for now since not in original data)
        weather_df['precipitation'] = np.random.exponential(1.5, len(weather_df))
    
    return weather_df

def load_all_regions_data():
    """
    Load data from all STAC cache files representing different regions.
    Returns combined dataset with region information for multi-region training.
    """
    cache_files = list_stac_cache_files()
    
    if not cache_files:
        print("⚠️  No STAC cache files found, using DEFAULT_BBOX")
        return pd.DataFrame(), [DEFAULT_BBOX], ["default_region"]
    
    all_regions_data = []
    all_bboxes = []
    region_names = []
    
    print(f"🗺️  Loading data from {len(cache_files)} regions...")
    
    for i, cache_file in enumerate(cache_files):
        try:
            # Load cached data with region info
            cached_df, region_info, bbox = load_cached_data_with_region(cache_file)
            
            if not cached_df.empty and bbox and len(bbox) == 4:
                # Add region identifier to the data
                cached_df = cached_df.copy()
                cached_df['region_id'] = f"region_{i+1}"
                cached_df['region_file'] = cache_file.name
                cached_df['bbox_min_lon'] = bbox[0]
                cached_df['bbox_min_lat'] = bbox[1]
                cached_df['bbox_max_lon'] = bbox[2]
                cached_df['bbox_max_lat'] = bbox[3]
                
                # Calculate region center for weather data
                cached_df['region_center_lat'] = (bbox[1] + bbox[3]) / 2
                cached_df['region_center_lon'] = (bbox[0] + bbox[2]) / 2
                
                # Add region info if available
                if region_info:
                    cached_df['region_area_km2'] = region_info.get('area_km2', 0)
                    cached_df['region_width_deg'] = region_info.get('width_deg', 0)
                    cached_df['region_height_deg'] = region_info.get('height_deg', 0)
                
                all_regions_data.append(cached_df)
                all_bboxes.append(bbox)
                region_names.append(cache_file.stem)
                
                print(f"   ✅ Region {i+1} ({cache_file.name}): {len(cached_df)} records")
                print(f"      Center: ({(bbox[0] + bbox[2])/2:.4f}, {(bbox[1] + bbox[3])/2:.4f})")
                
                if region_info:
                    print(f"      Area: {region_info.get('area_km2', 0):.1f} km²")
                    
            else:
                print(f"   ❌ Skipping {cache_file.name}: Invalid data or bbox")
                
        except Exception as e:
            print(f"   ⚠️  Error loading {cache_file.name}: {e}")
    
    if all_regions_data:
        # Combine all regions into single DataFrame
        combined_df = pd.concat(all_regions_data, ignore_index=True)
        combined_df = combined_df.sort_values('datetime').reset_index(drop=True)
        
        print(f"\n🌍 Multi-region dataset summary:")
        print(f"   Total regions: {len(all_bboxes)}")
        print(f"   Total records: {len(combined_df)}")
        print(f"   Date range: {combined_df['datetime'].min().date()} to {combined_df['datetime'].max().date()}")
        print(f"   Regions: {', '.join(region_names)}")
        
        # Display region statistics
        region_stats = combined_df.groupby('region_id').agg({
            'datetime': ['count', 'min', 'max'],
            'region_center_lat': 'first',
            'region_center_lon': 'first',
            'region_area_km2': 'first'
        }).round(4)
        
        print(f"\n📊 Per-region statistics:")
        for region_id in combined_df['region_id'].unique():
            region_data = combined_df[combined_df['region_id'] == region_id]
            print(f"   {region_id}: {len(region_data)} records, "
                  f"center ({region_data['region_center_lat'].iloc[0]:.4f}, "
                  f"{region_data['region_center_lon'].iloc[0]:.4f})")
        
        return combined_df, all_bboxes, region_names
    else:
        print("❌ No valid regions found, using default bbox")
        return pd.DataFrame(), [DEFAULT_BBOX], ["default_region"]

# Load multi-region data
try:
    MULTI_REGION_DATA, ALL_BBOXES, REGION_NAMES = load_all_regions_data()

    if not MULTI_REGION_DATA.empty:
        print(f"\n🎯 Multi-region training enabled with {len(ALL_BBOXES)} regions")
        # Use the first region's center as primary coordinates for fallback weather data
        PRIMARY_BBOX = ALL_BBOXES[0] if ALL_BBOXES else DEFAULT_BBOX
    else:
        print(f"\n⚠️  Falling back to single region mode")
        PRIMARY_BBOX = DEFAULT_BBOX

    print(f"\n🗺️  Primary region coordinates for weather fallback:")
    print(f"   Bounding box: {PRIMARY_BBOX}")
    print(f"   Center: ({(PRIMARY_BBOX[0] + PRIMARY_BBOX[2])/2:.4f}, {(PRIMARY_BBOX[1] + PRIMARY_BBOX[3])/2:.4f})")
    
except Exception as e:
    print(f"❌ Error in load_all_regions_data: {e}")
    print("   Using fallback values...")
    MULTI_REGION_DATA = pd.DataFrame()
    ALL_BBOXES = [DEFAULT_BBOX]
    REGION_NAMES = ["default_region"]
    PRIMARY_BBOX = DEFAULT_BBOX

🗺️  Loading data from 4 regions...
   ✅ Region 1 (53dc11e66dc23dff069f3e80716ccd7d0fc4d0836241e8ff753ad6ca2664048f.json): 241 records
      Center: (-76.8799, 39.0192)
      Area: 0.2 km²
   ✅ Region 2 (8c210d4a0d2ba8d3baadefd61f2d468d27500725b024bba68ad5e3691ede7f8c.json): 175 records
      Center: (31.1629, 36.8382)
      Area: 0.0 km²
   ✅ Region 3 (bafc2299e1379f32c12f9da1acd9a1e42f74c4872e306d410e04af5e1d859202.json): 155 records
      Center: (39.8385, 44.9317)
      Area: 4.7 km²
   ✅ Region 4 (c405874584da6e3b8d9bd4fbdc02984bccc68e4a89b790afdca3205a6e184124.json): 499 records
      Center: (-113.7456, 42.7059)
      Area: 778.8 km²

🌍 Multi-region dataset summary:
   Total regions: 4
   Total records: 1070
   Date range: 2020-04-20 to 2025-10-03
   Regions: 53dc11e66dc23dff069f3e80716ccd7d0fc4d0836241e8ff753ad6ca2664048f, 8c210d4a0d2ba8d3baadefd61f2d468d27500725b024bba68ad5e3691ede7f8c, bafc2299e1379f32c12f9da1acd9a1e42f74c4872e306d410e04af5e1d859202, c405874584da6e3b8d9bd4fbdc

In [25]:
# Shared utility functions for growing season analysis
def add_growing_season_indicator(df, date_column='ds'):
    """
    Add growing season indicator to dataframe.
    Growing season is defined as April through October (months 4-10) for Northern Hemisphere.
    
    Parameters:
    df (pd.DataFrame): DataFrame with datetime column
    date_column (str): Name of the datetime column (default: 'ds')
    
    Returns:
    pd.DataFrame: DataFrame with added 'is_growing_season' column
    """
    df = df.copy()
    df['is_growing_season'] = (
        (df[date_column].dt.month >= 4) & 
        (df[date_column].dt.month <= 10)
    )
    return df

def is_growing_season_mask(dates):
    """
    Create a boolean mask for growing season dates.
    
    Parameters:
    dates (pd.Series or pd.DatetimeIndex): Datetime series/index
    
    Returns:
    pd.Series: Boolean mask where True indicates growing season
    """
    return (dates.dt.month >= 4) & (dates.dt.month <= 10)

print("✅ Growing season utility functions defined")

✅ Growing season utility functions defined


In [26]:
def fetch_weather_forecast_data(lat, lon, start_date, end_date):
    """
    Fetch weather forecast data from Open-Meteo forecast API for future dates
    
    Parameters:
    lat (float): Latitude
    lon (float): Longitude 
    start_date (datetime.date): Start date for forecast
    end_date (datetime.date): End date for forecast
    
    Returns:
    pd.DataFrame: Weather forecast data
    """
    try:
        import requests
        
        # Open-Meteo forecast API (free, no API key required)
        # Note: Free API typically provides 7-16 days of forecast
        forecast_url = "https://api.open-meteo.com/v1/forecast"
        
        # Limit forecast period to maximum available (15 days to be safe)
        today = datetime.now().date()
        max_forecast_end = today + timedelta(days=15)  # More conservative limit
        limited_end_date = min(end_date, max_forecast_end)
        
        # Only proceed if there are forecast days available
        if start_date > limited_end_date:
            print(f"   Forecast period beyond API limit (15 days), using climatology...")
            return pd.DataFrame()
        
        # Parameters for forecast API - using correct Open-Meteo parameter names
        params = {
            'latitude': lat,
            'longitude': lon,
            'start_date': start_date.strftime('%Y-%m-%d'),
            'end_date': limited_end_date.strftime('%Y-%m-%d'),
            'daily': 'temperature_2m_max,temperature_2m_min,relative_humidity_2m_mean,precipitation_sum,wind_speed_10m_max',
            'timezone': 'auto'
        }
        
        print(f"   Fetching forecast from {forecast_url}")
        print(f"   Coordinates: ({lat:.4f}, {lon:.4f})")
        print(f"   Date range: {start_date} to {limited_end_date} (limited to 15 days)")
        
        response = requests.get(forecast_url, params=params, timeout=30)
        response.raise_for_status()
        
        data = response.json()
        
        if 'daily' not in data:
            print("   No daily forecast data available")
            return pd.DataFrame()
        
        # Parse forecast data
        daily_data = data['daily']
        
        # Calculate mean temperature from max and min
        temp_max = daily_data.get('temperature_2m_max', [])
        temp_min = daily_data.get('temperature_2m_min', [])
        temp_mean = [(tmax + tmin) / 2 if tmax is not None and tmin is not None else None 
                     for tmax, tmin in zip(temp_max, temp_min)]
        
        forecast_df = pd.DataFrame({
            'date': pd.to_datetime(daily_data['time']),
            'temperature_mean': temp_mean,
            'humidity': daily_data.get('relative_humidity_2m_mean', []),
            'precipitation': daily_data.get('precipitation_sum', []),
            'wind_speed_mean': daily_data.get('wind_speed_10m_max', [])  # Use max as proxy for mean
        })
        
        # Calculate derived weather variables
        if not forecast_df.empty:
            # Growing Degree Days (base 10°C)
            forecast_df['growing_degree_days'] = forecast_df['temperature_mean'].apply(
                lambda x: max(0, x - 10) if pd.notna(x) else 0
            )
            
            # Vapor Pressure Deficit (simplified calculation)
            forecast_df['vapor_pressure_deficit'] = forecast_df.apply(
                lambda row: calculate_vpd(row['temperature_mean'], row['humidity']) 
                if pd.notna(row['temperature_mean']) and pd.notna(row['humidity']) else 0,
                axis=1
            )
            
            # Add cloudcover and clarity index (estimates based on precipitation)
            forecast_df['cloudcover_mean'] = forecast_df['precipitation'].apply(
                lambda x: min(80, 30 + x * 10) if pd.notna(x) else 50  # More precip = more clouds
            )
            
            forecast_df['clarity_index'] = forecast_df['cloudcover_mean'].apply(
                lambda x: max(0.2, 1 - x/100) if pd.notna(x) else 0.6
            )
        
        print(f"   ✅ Retrieved forecast for {len(forecast_df)} days")
        
        # If the original request was for longer than available forecast, note this
        if end_date > limited_end_date:
            remaining_days = (end_date - limited_end_date).days
            print(f"   ⚠️  {remaining_days} days beyond forecast limit, will use climatology for remainder")
            
        return forecast_df
        
    except requests.exceptions.RequestException as e:
        print(f"   ❌ Forecast API request failed: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"   ❌ Error processing forecast data: {e}")
        return pd.DataFrame()

## Load Cached NDVI and Weather Data

This section loads historical NDVI data from the cache directory and aggregates it across multiple years for training.

In [27]:
def load_stac_cache_data():
    """Load all STAC cache files to get datetime information - now supports multi-region"""
    
    # Use the multi-region data if available
    if not MULTI_REGION_DATA.empty:
        print(f"🌍 Using multi-region cached data: {len(MULTI_REGION_DATA)} total records")
        stac_df = MULTI_REGION_DATA.copy()
        
        # Ensure proper column naming for compatibility
        if 'id' not in stac_df.columns and 'scene_id' in stac_df.columns:
            stac_df['id'] = stac_df['scene_id']
        
        return stac_df
    
    # Fallback to original single-region loading
    stac_data = []
    
    if not STAC_CACHE_DIR.exists():
        print("❌ STAC cache directory not found. Run the main app first to generate cache.")
        return pd.DataFrame()
    
    cache_files = list(STAC_CACHE_DIR.glob("*.json"))
    print(f"📁 Found {len(cache_files)} STAC cache files")
    
    for cache_file in cache_files:
        try:
            with open(cache_file, 'r') as f:
                data = json.load(f)
            
            records = data.get('records', [])
            for record in records:
                if record.get('datetime'):
                    stac_data.append({
                        'datetime': pd.to_datetime(record['datetime']),
                        'id': record.get('id'),
                        'cloud_cover': record.get('cloud_cover'),
                        'collection': record.get('collection'),
                        'cache_file': cache_file.name,
                        'region_id': 'single_region',
                        'region_file': cache_file.name
                    })
        except Exception as e:
            print(f"⚠️  Error loading {cache_file.name}: {e}")
    
    if stac_data:
        df = pd.DataFrame(stac_data)
        df = df.sort_values('datetime').reset_index(drop=True)
        print(f"✅ Loaded {len(df)} STAC records from {df['datetime'].min().date()} to {df['datetime'].max().date()}")
        return df
    else:
        print("❌ No valid STAC data found")
        return pd.DataFrame()

# Load STAC cache data (now multi-region aware)
stac_df = load_stac_cache_data()

if not stac_df.empty:
    print(f"📊 Multi-region STAC dataset loaded:")
    print(f"   Date range: {stac_df['datetime'].min().strftime('%Y-%m-%d')} to {stac_df['datetime'].max().strftime('%Y-%m-%d')}")
    
    if 'collection' in stac_df.columns:
        print(f"   Collections: {stac_df['collection'].value_counts().to_dict()}")
    
    if 'region_id' in stac_df.columns:
        region_counts = stac_df['region_id'].value_counts()
        print(f"   Records per region: {region_counts.to_dict()}")
        print(f"   Total regions: {len(region_counts)}")
    
else:
    print("❌ No STAC data available. Please run the main application first to generate cache data.")

🌍 Using multi-region cached data: 1070 total records
📊 Multi-region STAC dataset loaded:
   Date range: 2020-04-20 to 2025-10-03
   Collections: {'HLSS30_2.0': 675, 'HLSL30_2.0': 395}
   Records per region: {'region_4': 499, 'region_1': 241, 'region_2': 175, 'region_3': 155}
   Total regions: 4


In [28]:
def load_ndvi_cache_data():
    """Load cached NDVI data from .npz files"""
    ndvi_data = []
    
    if not NDVI_CACHE_DIR.exists():
        print("❌ NDVI cache directory not found.")
        return pd.DataFrame()
    
    cache_files = list(NDVI_CACHE_DIR.glob("*.npz"))
    print(f"📁 Found {len(cache_files)} NDVI cache files")
    
    for cache_file in cache_files:
        try:
            with np.load(cache_file, allow_pickle=False) as data:
                ndvi_array = data['NDVI']
                mask = data['mask']
                mean_ndvi = float(data['mean'].item())
                
                # Create masked array and compute statistics
                masked_ndvi = np.ma.array(ndvi_array, mask=mask)
                
                if masked_ndvi.count() > 0:
                    ndvi_data.append({
                        'cache_file': cache_file.name,
                        'mean_ndvi': mean_ndvi,
                        'std_ndvi': float(masked_ndvi.std()) if masked_ndvi.count() > 1 else 0,
                        'min_ndvi': float(masked_ndvi.min()),
                        'max_ndvi': float(masked_ndvi.max()),
                        'valid_pixels': int(masked_ndvi.count()),
                        'crop_pixels': int(np.sum(masked_ndvi.compressed() >= 0.35)),  # CROP_NDVI_THRESHOLD
                    })
        except Exception as e:
            print(f"⚠️  Error loading {cache_file.name}: {e}")
    
    if ndvi_data:
        df = pd.DataFrame(ndvi_data)
        print(f"✅ Loaded {len(df)} NDVI cache files")
        print(f"📊 Mean NDVI range: {df['mean_ndvi'].min():.3f} to {df['mean_ndvi'].max():.3f}")
        return df
    else:
        print("❌ No valid NDVI cache data found")
        return pd.DataFrame()

# Load NDVI cache data
ndvi_cache_df = load_ndvi_cache_data()

📁 Found 1178 NDVI cache files


✅ Loaded 1178 NDVI cache files
📊 Mean NDVI range: -0.070 to 0.873


In [29]:
# Load weather data for all regions
def load_weather_for_all_regions(stac_df):
    """Load weather data for each region in the dataset"""
    if stac_df.empty:
        return pd.DataFrame()
    
    all_weather_data = []
    
    # Get unique regions and their coordinates
    if 'region_center_lat' in stac_df.columns and 'region_center_lon' in stac_df.columns:
        # Multi-region mode
        unique_regions = stac_df[['region_id', 'region_center_lat', 'region_center_lon']].drop_duplicates()
        
        print(f"🌡️  Fetching weather data for {len(unique_regions)} regions...")
        
        start_date = stac_df['datetime'].min().date()
        end_date = min(stac_df['datetime'].max().date(), datetime.now().date())
        
        for _, region in unique_regions.iterrows():
            region_id = region['region_id']
            lat = region['region_center_lat']
            lon = region['region_center_lon']
            
            print(f"   🌍 {region_id}: ({lat:.4f}, {lon:.4f})")
            
            try:
                weather_df = fetch_weather_data_for_period(lat, lon, start_date, end_date)
                
                if not weather_df.empty:
                    weather_df['region_id'] = region_id
                    weather_df['region_lat'] = lat
                    weather_df['region_lon'] = lon
                    all_weather_data.append(weather_df)
                    
                    print(f"      ✅ Loaded {len(weather_df)} weather records")
                else:
                    print(f"      ❌ No weather data available")
                    
            except Exception as e:
                print(f"      ⚠️  Weather fetch failed: {e}")
    
    else:
        # Single region fallback
        center_lat = (PRIMARY_BBOX[1] + PRIMARY_BBOX[3]) / 2
        center_lon = (PRIMARY_BBOX[0] + PRIMARY_BBOX[2]) / 2
        
        print(f"🌡️  Fetching weather data for primary region: {center_lat:.4f}, {center_lon:.4f}")
        
        start_date = stac_df['datetime'].min().date()
        end_date = min(stac_df['datetime'].max().date(), datetime.now().date())
        
        weather_df = fetch_weather_data_for_period(center_lat, center_lon, start_date, end_date)
        
        if not weather_df.empty:
            weather_df['region_id'] = 'single_region'
            weather_df['region_lat'] = center_lat
            weather_df['region_lon'] = center_lon
            all_weather_data.append(weather_df)
    
    if all_weather_data:
        combined_weather = pd.concat(all_weather_data, ignore_index=True)
        
        print(f"✅ Combined weather dataset: {len(combined_weather)} records")
        print(f"   Date range: {combined_weather['date'].min()} to {combined_weather['date'].max()}")
        print(f"   Regions with weather data: {combined_weather['region_id'].nunique()}")
        print(f"   Weather features: {list(combined_weather.columns[~combined_weather.columns.isin(['date', 'region_id', 'region_lat', 'region_lon'])])}")
        
        return combined_weather
    else:
        print("❌ No weather data loaded for any region")
        return pd.DataFrame()

# Load weather data for all regions
weather_df = load_weather_for_all_regions(stac_df)

if not weather_df.empty:
    # Display weather summary by region
    print(f"\n📊 Weather data summary by region:")
    for region_id in weather_df['region_id'].unique():
        region_weather = weather_df[weather_df['region_id'] == region_id]
        print(f"   {region_id}: {len(region_weather)} records, "
              f"temp range {region_weather['temperature_mean'].min():.1f}°C to {region_weather['temperature_mean'].max():.1f}°C")
else:
    print("⚠️  No weather data available - will use synthetic data during preprocessing")

🌡️  Fetching weather data for 4 regions...
   🌍 region_4: (42.7059, -113.7456)
      ✅ Loaded 1993 weather records
   🌍 region_1: (39.0192, -76.8799)
      ✅ Loaded 1993 weather records
   🌍 region_2: (36.8382, 31.1629)
      ✅ Loaded 1993 weather records
   🌍 region_3: (44.9317, 39.8385)
      ✅ Loaded 1993 weather records
✅ Combined weather dataset: 7972 records
   Date range: 2020-04-20 00:00:00 to 2025-10-03 00:00:00
   Regions with weather data: 4
   Weather features: ['temperature_mean', 'humidity', 'cloudcover_mean', 'wind_speed_mean', 'clarity_index', 'vapor_pressure_deficit', 'growing_degree_days', 'precipitation']

📊 Weather data summary by region:
   region_4: 1993 records, temp range -15.0°C to 31.7°C
   region_1: 1993 records, temp range -10.7°C to 32.5°C
   region_2: 1993 records, temp range 2.7°C to 35.8°C
   region_3: 1993 records, temp range -14.7°C to 32.6°C


## Data Preprocessing and Feature Engineering

Now we'll combine the NDVI and weather data, handle missing values, and create features for the Prophet model.

In [30]:
def create_synthetic_ndvi_data():
    """Create synthetic NDVI data for demonstration if no cache is available"""
    print("📝 Creating synthetic NDVI data for demonstration...")
    
    # Create 3 years of synthetic data
    start_date = datetime(CURRENT_YEAR - 3, 1, 1)
    end_date = datetime(CURRENT_YEAR - 1, 12, 31)
    
    date_range = pd.date_range(start_date, end_date, freq='16D')  # Similar to satellite revisit
    
    synthetic_data = []
    for date in date_range:
        # Create seasonal NDVI pattern with some noise
        day_of_year = date.timetuple().tm_yday
        
        # Base seasonal pattern (higher in summer, lower in winter)
        seasonal = 0.4 + 0.3 * np.sin(2 * np.pi * (day_of_year - 90) / 365)
        
        # Add some randomness
        noise = np.random.normal(0, 0.05)
        
        # Add weather influence (synthetic)
        temp_effect = np.random.normal(0, 0.02)
        
        ndvi = np.clip(seasonal + noise + temp_effect, 0, 1)
        
        synthetic_data.append({
            'datetime': date,
            'mean_ndvi': ndvi,
            'std_ndvi': np.random.uniform(0.02, 0.08),
            'valid_pixels': np.random.randint(8000, 12000),
            'crop_pixels': int(np.random.uniform(0.3, 0.8) * np.random.randint(8000, 12000)),
            'region_id': 'synthetic_region'
        })
    
    return pd.DataFrame(synthetic_data)

# Prepare NDVI time series - now multi-region aware
if not stac_df.empty:
    print("🔄 Processing multi-region STAC data for NDVI time series...")
    
    # Check if we have actual NDVI values or need to create them
    if 'mean_ndvi' in stac_df.columns:
        # Use existing NDVI data
        ndvi_ts = stac_df[['datetime', 'mean_ndvi', 'region_id']].copy()
        
        # Add crop fraction if not available
        if 'crop_fraction' not in ndvi_ts.columns:
            ndvi_ts['crop_fraction'] = np.random.uniform(0.2, 0.8, len(ndvi_ts))
            
        print(f"✅ Using cached NDVI data: {len(ndvi_ts)} observations across {ndvi_ts['region_id'].nunique()} regions")
        
    else:
        # Create synthetic NDVI values distributed across real timestamps and regions
        print("📝 Creating synthetic NDVI values for multi-region STAC data...")
        
        ndvi_ts = stac_df.copy()
        
        # Generate region-specific NDVI patterns
        for region_id in ndvi_ts['region_id'].unique():
            region_mask = ndvi_ts['region_id'] == region_id
            region_dates = ndvi_ts.loc[region_mask, 'datetime']
            
            # Create slightly different seasonal patterns per region
            region_ndvi = []
            for date in region_dates:
                day_of_year = date.timetuple().tm_yday
                
                # Base seasonal pattern with region variation
                region_offset = hash(region_id) % 100 / 1000  # Small region-specific offset
                seasonal = 0.4 + region_offset + 0.3 * np.sin(2 * np.pi * (day_of_year - 90) / 365)
                
                # Add noise
                noise = np.random.normal(0, 0.05)
                
                ndvi = np.clip(seasonal + noise, 0, 1)
                region_ndvi.append(ndvi)
            
            ndvi_ts.loc[region_mask, 'mean_ndvi'] = region_ndvi
        
        # Add crop fraction
        ndvi_ts['crop_fraction'] = np.random.uniform(0.2, 0.8, len(ndvi_ts))
        
        # Keep only relevant columns
        ndvi_ts = ndvi_ts[['datetime', 'mean_ndvi', 'crop_fraction', 'region_id']].copy()
        
        print(f"✅ Generated synthetic NDVI for {len(ndvi_ts)} observations across {ndvi_ts['region_id'].nunique()} regions")
        
else:
    print("⚠️  No STAC data available, creating fully synthetic multi-region dataset...")
    
    # Create synthetic data for multiple mock regions
    all_synthetic = []
    mock_regions = ['region_1', 'region_2', 'region_3']
    
    for region_id in mock_regions:
        region_data = create_synthetic_ndvi_data()
        region_data['region_id'] = region_id
        all_synthetic.append(region_data)
    
    ndvi_ts = pd.concat(all_synthetic, ignore_index=True)
    ndvi_ts['crop_fraction'] = ndvi_ts['crop_pixels'] / ndvi_ts['valid_pixels']

# Ensure datetime column and sort
ndvi_ts['datetime'] = pd.to_datetime(ndvi_ts['datetime'])
ndvi_ts = ndvi_ts.sort_values(['region_id', 'datetime']).reset_index(drop=True)

print(f"\n🌍 Multi-region NDVI time series prepared:")
print(f"   Total observations: {len(ndvi_ts)}")
print(f"   Number of regions: {ndvi_ts['region_id'].nunique()}")
print(f"   Date range: {ndvi_ts['datetime'].min().date()} to {ndvi_ts['datetime'].max().date()}")
print(f"   NDVI range: {ndvi_ts['mean_ndvi'].min():.3f} to {ndvi_ts['mean_ndvi'].max():.3f}")

# Display per-region statistics
print(f"\n📊 NDVI statistics by region:")
region_stats = ndvi_ts.groupby('region_id')['mean_ndvi'].agg(['count', 'mean', 'std', 'min', 'max']).round(3)
for region_id, stats in region_stats.iterrows():
    print(f"   {region_id}: {stats['count']} obs, mean={stats['mean']:.3f}±{stats['std']:.3f}, range=[{stats['min']:.3f}, {stats['max']:.3f}]")

# Display sample data
print(f"\n📋 Sample multi-region NDVI data:")
print(ndvi_ts.head(10))

🔄 Processing multi-region STAC data for NDVI time series...
📝 Creating synthetic NDVI values for multi-region STAC data...
✅ Generated synthetic NDVI for 1070 observations across 4 regions

🌍 Multi-region NDVI time series prepared:
   Total observations: 1070
   Number of regions: 4
   Date range: 2020-04-20 to 2025-10-03
   NDVI range: 0.058 to 0.873

📊 NDVI statistics by region:
   region_1: 241.0 obs, mean=0.470±0.211, range=[0.063, 0.838]
   region_2: 175.0 obs, mean=0.486±0.204, range=[0.058, 0.832]
   region_3: 155.0 obs, mean=0.530±0.206, range=[0.073, 0.873]
   region_4: 499.0 obs, mean=0.526±0.196, range=[0.065, 0.859]

📋 Sample multi-region NDVI data:
                          datetime  mean_ndvi  crop_fraction region_id
0 2020-06-07 16:12:21.944000+00:00   0.755169       0.610878  region_1
1 2020-06-09 16:02:30.159000+00:00   0.763070       0.317581  region_1
2 2020-06-24 16:02:26.381000+00:00   0.837310       0.721335  region_1
3 2020-06-29 16:02:29.019000+00:00   0.797164 

In [31]:
# Merge multi-region NDVI with weather data
if not weather_df.empty:
    print("🔄 Merging multi-region NDVI with weather data...")
    
    # Convert to date for merging
    ndvi_ts['date'] = ndvi_ts['datetime'].dt.date
    weather_df['date'] = pd.to_datetime(weather_df['date']).dt.date
    
    # Merge on both date and region_id
    merged_df = pd.merge(
        ndvi_ts, 
        weather_df, 
        on=['date', 'region_id'], 
        how='left'
    )
    
    # Fill missing weather data with interpolation within each region
    weather_cols = ['temperature_mean', 'humidity', 'precipitation', 'growing_degree_days', 'vapor_pressure_deficit']
    
    for region_id in merged_df['region_id'].unique():
        region_mask = merged_df['region_id'] == region_id
        
        for col in weather_cols:
            if col in merged_df.columns:
                # Interpolate within region, then fill with region median
                region_data = merged_df.loc[region_mask, col]
                merged_df.loc[region_mask, col] = (
                    region_data.interpolate(method='linear')
                    .fillna(region_data.median())
                    .fillna(merged_df[col].median())  # Global fallback
                )
    
    print(f"✅ Merged multi-region dataset: {len(merged_df)} rows across {merged_df['region_id'].nunique()} regions")
    
    # Display merge statistics
    merge_stats = merged_df.groupby('region_id').agg({
        'datetime': 'count',
        'temperature_mean': ['mean', 'std'],
        'mean_ndvi': ['mean', 'std']
    }).round(3)
    
    print(f"\n📊 Merged dataset statistics by region:")
    for region_id in merged_df['region_id'].unique():
        region_data = merged_df[merged_df['region_id'] == region_id]
        print(f"   {region_id}: {len(region_data)} records, "
              f"temp {region_data['temperature_mean'].mean():.1f}±{region_data['temperature_mean'].std():.1f}°C, "
              f"NDVI {region_data['mean_ndvi'].mean():.3f}±{region_data['mean_ndvi'].std():.3f}")
    
else:
    print("⚠️  No weather data available, creating synthetic weather features for all regions...")
    merged_df = ndvi_ts.copy()
    merged_df['date'] = merged_df['datetime'].dt.date
    
    # Create region-specific synthetic weather features
    np.random.seed(42)  # For reproducibility
    
    for region_id in merged_df['region_id'].unique():
        region_mask = merged_df['region_id'] == region_id
        region_data = merged_df[region_mask].copy()
        
        # Create region-specific climate patterns
        region_temp_offset = (hash(region_id) % 20 - 10) * 0.5  # ±5°C regional variation
        region_humidity_offset = (hash(region_id) % 30 - 15)     # ±15% humidity variation
        
        # Generate weather with regional characteristics
        merged_df.loc[region_mask, 'temperature_mean'] = (
            15 + region_temp_offset + 
            10 * np.sin(2 * np.pi * region_data['datetime'].dt.dayofyear / 365) + 
            np.random.normal(0, 2, len(region_data))
        )
        
        merged_df.loc[region_mask, 'humidity'] = (
            60 + region_humidity_offset + 
            20 * np.sin(2 * np.pi * (region_data['datetime'].dt.dayofyear - 180) / 365) + 
            np.random.normal(0, 5, len(region_data))
        )
        
        merged_df.loc[region_mask, 'precipitation'] = np.maximum(
            0, np.random.exponential(2, len(region_data))
        )
        
        merged_df.loc[region_mask, 'growing_degree_days'] = np.maximum(
            0, merged_df.loc[region_mask, 'temperature_mean'] - 10
        )
        
        merged_df.loc[region_mask, 'vapor_pressure_deficit'] = calculate_vpd(
            merged_df.loc[region_mask, 'temperature_mean'], 
            merged_df.loc[region_mask, 'humidity']
        )
    
    print(f"✅ Created synthetic weather for {merged_df['region_id'].nunique()} regions")

# Feature engineering - now region-aware
merged_df['month'] = merged_df['datetime'].dt.month
merged_df['day_of_year'] = merged_df['datetime'].dt.dayofyear
merged_df['season'] = merged_df['month'].apply(
    lambda x: 'Spring' if x in [3,4,5] else 
              'Summer' if x in [6,7,8] else 
              'Fall' if x in [9,10,11] else 
              'Winter'
)

# Temperature anomalies calculated within each region
merged_df['temp_anomaly'] = (
    merged_df['temperature_mean'] - 
    merged_df.groupby(['region_id', 'month'])['temperature_mean'].transform('mean')
)

# Cumulative GDD per region per year
merged_df['cumulative_gdd'] = merged_df.groupby(['region_id', merged_df['datetime'].dt.year])['growing_degree_days'].cumsum()

# Lag features within each region
for region_id in merged_df['region_id'].unique():
    region_mask = merged_df['region_id'] == region_id
    region_data = merged_df[region_mask].sort_values('datetime')
    
    merged_df.loc[region_mask, 'ndvi_lag_1'] = region_data['mean_ndvi'].shift(1)
    merged_df.loc[region_mask, 'ndvi_lag_2'] = region_data['mean_ndvi'].shift(2)
    merged_df.loc[region_mask, 'temp_lag_1'] = region_data['temperature_mean'].shift(1)

# Add region-specific features
merged_df['region_numeric'] = pd.Categorical(merged_df['region_id']).codes

# Remove rows with NaN values from lag features
modeling_df = merged_df.dropna().reset_index(drop=True)

print(f"\n🔧 Multi-region feature engineering complete:")
print(f"   Total rows after cleaning: {len(modeling_df)}")
print(f"   Regions: {modeling_df['region_id'].nunique()}")
print(f"   Features available: {list(modeling_df.columns)}")

# Display correlation with NDVI by region
weather_features = ['temperature_mean', 'humidity', 'growing_degree_days', 'vapor_pressure_deficit', 'precipitation']
available_features = [col for col in weather_features if col in modeling_df.columns]

if available_features:
    print(f"\n🔍 Feature correlations with NDVI by region:")
    
    for region_id in modeling_df['region_id'].unique():
        region_data = modeling_df[modeling_df['region_id'] == region_id]
        if len(region_data) > 1:
            correlations = region_data[available_features + ['mean_ndvi']].corr()['mean_ndvi'].drop('mean_ndvi')
            print(f"\n   {region_id} ({len(region_data)} observations):")
            for feature, corr in correlations.items():
                print(f"     {feature}: {corr:.3f}")

    # Overall correlations across all regions
    print(f"\n   Overall (all regions combined):")
    overall_correlations = modeling_df[available_features + ['mean_ndvi']].corr()['mean_ndvi'].drop('mean_ndvi')
    for feature, corr in overall_correlations.items():
        print(f"     {feature}: {corr:.3f}")

print(f"\n🌍 Multi-region dataset ready for Prophet training!")

🔄 Merging multi-region NDVI with weather data...
✅ Merged multi-region dataset: 1070 rows across 4 regions

📊 Merged dataset statistics by region:
   region_1: 241 records, temp 14.0±9.6°C, NDVI 0.470±0.211
   region_2: 175 records, temp 21.2±7.7°C, NDVI 0.486±0.204
   region_3: 155 records, temp 18.4±10.1°C, NDVI 0.530±0.206
   region_4: 499 records, temp 15.9±10.1°C, NDVI 0.526±0.196

🔧 Multi-region feature engineering complete:
   Total rows after cleaning: 1062
   Regions: 4
   Features available: ['datetime', 'mean_ndvi', 'crop_fraction', 'region_id', 'date', 'temperature_mean', 'humidity', 'cloudcover_mean', 'wind_speed_mean', 'clarity_index', 'vapor_pressure_deficit', 'growing_degree_days', 'precipitation', 'region_lat', 'region_lon', 'month', 'day_of_year', 'season', 'temp_anomaly', 'cumulative_gdd', 'ndvi_lag_1', 'ndvi_lag_2', 'temp_lag_1', 'region_numeric']

🔍 Feature correlations with NDVI by region:

   region_1 (239 observations):
     temperature_mean: 0.842
     humidity

## Prepare Data for Prophet Model

Format the data according to Prophet's requirements (ds, y columns) and prepare additional regressors.

In [32]:
# Prepare Prophet dataset - now with multi-region support
prophet_df = modeling_df[['datetime', 'mean_ndvi', 'region_id']].copy()
prophet_df.columns = ['ds', 'y', 'region_id']  # Prophet requires 'ds' and 'y' columns

# Remove timezone from ds column (Prophet doesn't support timezones)
prophet_df['ds'] = prophet_df['ds'].dt.tz_localize(None)

# Add regressors (weather variables)
regressors = ['temperature_mean', 'humidity', 'growing_degree_days', 'vapor_pressure_deficit', 'precipitation']
available_regressors = [col for col in regressors if col in modeling_df.columns]

for regressor in available_regressors:
    prophet_df[regressor] = modeling_df[regressor]

# Add region as a categorical regressor
prophet_df['region_numeric'] = modeling_df['region_numeric']

# Add region-specific seasonal indicators
prophet_df['is_northern_region'] = (
    modeling_df['region_id'].str.contains('north', case=False, na=False) |
    (modeling_df.get('region_center_lat', 0) > 40)  # Latitude-based if available
).astype(int)

# Split data for training and validation - maintain region distribution
split_date = prophet_df['ds'].max() - pd.DateOffset(months=6)

# Stratified split to ensure all regions are represented in both sets
train_df = prophet_df[prophet_df['ds'] <= split_date].copy()
val_df = prophet_df[prophet_df['ds'] > split_date].copy()

# Ensure we have data from all regions in training set
train_regions = set(train_df['region_id'].unique())
all_regions = set(prophet_df['region_id'].unique())

if len(train_regions) < len(all_regions):
    print("⚠️  Some regions missing from training set, adjusting split...")
    # Include at least some data from each region in training
    min_train_samples = 5  # Minimum samples per region for training
    
    adjusted_train_data = []
    adjusted_val_data = []
    
    for region_id in all_regions:
        region_data = prophet_df[prophet_df['region_id'] == region_id].sort_values('ds')
        
        if len(region_data) > min_train_samples:
            # Split this region's data
            region_split_idx = max(min_train_samples, len(region_data) - len(region_data) // 4)
            adjusted_train_data.append(region_data.iloc[:region_split_idx])
            if region_split_idx < len(region_data):
                adjusted_val_data.append(region_data.iloc[region_split_idx:])
        else:
            # Too few samples, put all in training
            adjusted_train_data.append(region_data)
    
    train_df = pd.concat(adjusted_train_data, ignore_index=True)
    if adjusted_val_data:
        val_df = pd.concat(adjusted_val_data, ignore_index=True)
    else:
        val_df = pd.DataFrame(columns=prophet_df.columns)

print(f"📊 Multi-region Prophet dataset prepared:")
print(f"   Total observations: {len(prophet_df)}")
print(f"   Total regions: {prophet_df['region_id'].nunique()}")
print(f"   Training set: {len(train_df)} observations ({train_df['ds'].min().date()} to {train_df['ds'].max().date()})")
print(f"   Validation set: {len(val_df)} observations ({val_df['ds'].min().date() if not val_df.empty else 'None'} to {val_df['ds'].max().date() if not val_df.empty else 'None'})")

# Display region distribution
print(f"\n📊 Region distribution:")
train_region_counts = train_df['region_id'].value_counts()
val_region_counts = val_df['region_id'].value_counts() if not val_df.empty else pd.Series()

print(f"   Training regions: {dict(train_region_counts)}")
if not val_region_counts.empty:
    print(f"   Validation regions: {dict(val_region_counts)}")

print(f"\n   Available regressors: {available_regressors + ['region_numeric', 'is_northern_region']}")

# Display training data statistics by region
print(f"\n📋 Training data summary by region:")
for region_id in train_df['region_id'].unique():
    region_data = train_df[train_df['region_id'] == region_id]
    print(f"\n   {region_id} ({len(region_data)} observations):")
    region_summary = region_data[['y'] + available_regressors].describe().round(3)
    print(f"     NDVI: mean={region_summary.loc['mean', 'y']:.3f}, std={region_summary.loc['std', 'y']:.3f}")
    if 'temperature_mean' in available_regressors:
        print(f"     Temp: mean={region_summary.loc['mean', 'temperature_mean']:.1f}°C, std={region_summary.loc['std', 'temperature_mean']:.1f}°C")

print(f"\n🌍 Multi-region dataset ready for Prophet model training!")

📊 Multi-region Prophet dataset prepared:
   Total observations: 1062
   Total regions: 4
   Training set: 840 observations (2020-05-08 to 2025-03-30)
   Validation set: 222 observations (2025-04-05 to 2025-10-03)

📊 Region distribution:
   Training regions: {'region_4': np.int64(422), 'region_1': np.int64(221), 'region_2': np.int64(116), 'region_3': np.int64(81)}
   Validation regions: {'region_4': np.int64(75), 'region_3': np.int64(72), 'region_2': np.int64(57), 'region_1': np.int64(18)}

   Available regressors: ['temperature_mean', 'humidity', 'growing_degree_days', 'vapor_pressure_deficit', 'precipitation', 'region_numeric', 'is_northern_region']

📋 Training data summary by region:

   region_1 (221 observations):
     NDVI: mean=0.452, std=0.210
     Temp: mean=13.2°C, std=9.5°C

   region_2 (116 observations):
     NDVI: mean=0.420, std=0.208
     Temp: mean=19.0°C, std=8.0°C

   region_3 (81 observations):
     NDVI: mean=0.433, std=0.219
     Temp: mean=14.3°C, std=11.8°C

   r

## Train Prophet Model with Weather Regressors

Initialize the Prophet model with appropriate parameters and fit it to the historical data.

In [33]:
# Initialize a Prophet model to properly view regressor coefficients
print("🔄 Reinitializing Prophet model for coefficient analysis...")

# Create a new Prophet model instance
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=10.0,
    n_changepoints=25
)

# Add regressors
print("📊 Adding regressors to model...")
all_regressors = available_regressors + ['region_numeric', 'is_northern_region']
for regressor in all_regressors:
    model.add_regressor(regressor, prior_scale=10.0, standardize=True)
    print(f"   Added regressor: {regressor}")

print(f"✅ Prophet model initialized with {len(all_regressors)} regressors")

🔄 Reinitializing Prophet model for coefficient analysis...
📊 Adding regressors to model...
   Added regressor: temperature_mean
   Added regressor: humidity
   Added regressor: growing_degree_days
   Added regressor: vapor_pressure_deficit
   Added regressor: precipitation
   Added regressor: region_numeric
   Added regressor: is_northern_region
✅ Prophet model initialized with 7 regressors


In [34]:
# Train the model and display ALL regressor coefficients
print("🔧 Training model with all regressors...")

try:
    model.fit(train_df)
    print("✅ Model training completed!")
    
    # Display ALL regressor coefficients properly
    if hasattr(model, 'params') and 'beta' in model.params:
        print(f"\n📊 Complete Regressor Coefficients:")
        
        beta_values = model.params['beta']
        print(f"   Beta array shape: {beta_values.shape}")
        print(f"   Number of regressors: {len(all_regressors)}")
        
        # Display each regressor coefficient
        for i, regressor in enumerate(all_regressors):
            if i < beta_values.shape[1]:
                # Extract coefficient for this regressor
                coeff_samples = beta_values[:, i]
                mean_coeff = coeff_samples.mean()
                std_coeff = coeff_samples.std()
                
                print(f"     {regressor:25}: {mean_coeff:8.4f} ± {std_coeff:6.4f}")
            else:
                print(f"     {regressor:25}: Index out of range")
        
        print(f"\n🎯 Coefficient Analysis:")
        
        # Sort regressors by absolute coefficient magnitude
        regressor_coeffs = []
        for i, regressor in enumerate(all_regressors):
            if i < beta_values.shape[1]:
                coeff = abs(beta_values[:, i].mean())
                regressor_coeffs.append((regressor, coeff))
        
        regressor_coeffs.sort(key=lambda x: x[1], reverse=True)
        
        print("   Regressors by importance (absolute coefficient):")
        for regressor, coeff in regressor_coeffs:
            print(f"     {regressor:25}: {coeff:.4f}")
            
    else:
        print("❌ No beta coefficients found in model parameters")
        
except Exception as e:
    print(f"❌ Model training failed: {e}")
    import traceback
    traceback.print_exc()

10:10:39 - cmdstanpy - INFO - Chain [1] start processing


🔧 Training model with all regressors...


10:10:39 - cmdstanpy - INFO - Chain [1] done processing


✅ Model training completed!

📊 Complete Regressor Coefficients:
   Beta array shape: (1, 27)
   Number of regressors: 7
     temperature_mean         :   0.0205 ± 0.0000
     humidity                 :  -0.3409 ± 0.0000
     growing_degree_days      :  -0.0013 ± 0.0000
     vapor_pressure_deficit   :  -0.0006 ± 0.0000
     precipitation            :   0.0020 ± 0.0000
     region_numeric           :   0.0052 ± 0.0000
     is_northern_region       :   0.0026 ± 0.0000

🎯 Coefficient Analysis:
   Regressors by importance (absolute coefficient):
     humidity                 : 0.3409
     temperature_mean         : 0.0205
     region_numeric           : 0.0052
     is_northern_region       : 0.0026
     precipitation            : 0.0020
     growing_degree_days      : 0.0013
     vapor_pressure_deficit   : 0.0006


## Generate Future Predictions

Create future dataframe and generate NDVI predictions for the current year.

In [35]:
if model is not None:
    # Create future dataframe starting from current timestamp
    current_time = datetime.now()
    prediction_start = datetime(current_time.year, 1, 1)
    prediction_end = current_time + pd.DateOffset(months=3)
    prediction_year = prediction_end.year
    
    # Create future dates (every 16 days to match satellite revisit cycle)
    future_dates = pd.date_range(
        start=prediction_start,
        end=prediction_end,
        freq='16D'
    )
    
    print(f"📅 Creating predictions from current time: {current_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"   Prediction period: 3 months ahead")
    print(f"   Prediction dates: {len(future_dates)} dates from {future_dates[0].date()} to {future_dates[-1].date()}")
    
    # Get weather forecasts/climatology for prediction period
    print("🌡️  Getting weather data for prediction period...")
    
    # Use the primary region's center coordinates for weather data
    primary_center_lat = (PRIMARY_BBOX[1] + PRIMARY_BBOX[3]) / 2
    primary_center_lon = (PRIMARY_BBOX[0] + PRIMARY_BBOX[2]) / 2
    
    # Use appropriate API based on date range
    today = datetime.now().date()
    current_weather = pd.DataFrame()
    
    if prediction_start.date() > today:
        # Future dates: Use forecast API
        print("   Prediction starts in future, using weather forecast API...")
        current_weather = fetch_weather_forecast_data(
            primary_center_lat, primary_center_lon, 
            prediction_start.date(), 
            prediction_end.date()
        )
    else:
        # Mixed dates: Get historical data and forecast data separately
        if prediction_end.date() > today:
            print("   Prediction spans past and future, combining historical + forecast...")
            
            # Get historical data up to today
            historical_weather = fetch_weather_data_for_period(
                primary_center_lat, primary_center_lon, 
                prediction_start.date(), 
                today
            )
            
            # Get forecast data from tomorrow onwards
            future_start = today + pd.DateOffset(days=1)
            forecast_weather = fetch_weather_forecast_data(
                primary_center_lat, primary_center_lon, 
                future_start.date(), 
                prediction_end.date()
            )
            
            # Combine historical and forecast data
            if not historical_weather.empty and not forecast_weather.empty:
                current_weather = pd.concat([historical_weather, forecast_weather], ignore_index=True)
            elif not historical_weather.empty:
                current_weather = historical_weather
            elif not forecast_weather.empty:
                current_weather = forecast_weather
        else:
            # Only historical dates: Use archive API
            print("   Using historical weather data...")
            current_weather = fetch_weather_data_for_period(
                primary_center_lat, primary_center_lon, 
                prediction_start.date(), 
                prediction_end.date()
            )
    
    if current_weather.empty:
        print("⚠️  No weather data available for prediction period, using climatology...")
        # Create climatological weather data based on historical means
        current_weather_list = []
        for date in future_dates:
            # Use day of year to create climatological values
            doy = date.timetuple().tm_yday
            
            # Seasonal patterns
            temp = 15 + 10 * np.sin(2 * np.pi * (doy - 90) / 365)
            humidity = 60 + 20 * np.sin(2 * np.pi * (doy - 180) / 365)
            
            current_weather_list.append({
                'date': date,
                'temperature_mean': temp,
                'humidity': humidity,
                'precipitation': 1.0,  # Climatological average
                'growing_degree_days': max(0, temp - 10),
                'vapor_pressure_deficit': calculate_vpd(temp, humidity)
            })
        
        current_weather = pd.DataFrame(current_weather_list)
        current_weather['date'] = pd.to_datetime(current_weather['date'])
    
    # Create future dataframe
    future_df = pd.DataFrame({'ds': future_dates})
    
    # Merge with weather data
    future_df['date'] = future_df['ds'].dt.date
    current_weather['date'] = current_weather['date'].dt.date
    
    future_with_weather = pd.merge(future_df, current_weather, on='date', how='left')
    
    # Fill missing weather values with interpolation
    for regressor in available_regressors:
        if regressor in current_weather.columns:
            future_with_weather[regressor] = future_with_weather[regressor].interpolate().fillna(
                current_weather[regressor].median()
            )
        else:
            # Use climatological values if regressor not available
            future_with_weather[regressor] = 0
    
    # Add region-based regressors that the model expects
    # Use the primary region's characteristics for predictions
    if len(ALL_BBOXES) > 0:
        # Use the first region (region_1) as default for predictions
        future_with_weather['region_numeric'] = 0  # First region
        
        # Check if it's a northern region based on latitude
        primary_lat = (PRIMARY_BBOX[1] + PRIMARY_BBOX[3]) / 2
        future_with_weather['is_northern_region'] = int(primary_lat > 40)
    else:
        # Fallback values
        future_with_weather['region_numeric'] = 0
        future_with_weather['is_northern_region'] = 0
    
    # Add growing season indicator (Prophet model expects all training columns)
    future_with_weather = add_growing_season_indicator(future_with_weather)
    
    print("🔮 Generating predictions with model...")
    print(f"   Using region_numeric: {future_with_weather['region_numeric'].iloc[0]}")
    print(f"   Using is_northern_region: {future_with_weather['is_northern_region'].iloc[0]}")
    print(f"   Available regressors: {list(future_with_weather.columns)}")
    
    try:
        # Make predictions using the model
        forecast = model.predict(future_with_weather)
        
        # Combine with historical data for complete timeline
        historical_forecast = model.predict(train_df)
        
        print(f"✅ Predictions generated for {len(forecast)} future dates")
        print(f"📊 Predicted NDVI range: {forecast['yhat'].min():.3f} to {forecast['yhat'].max():.3f}")
        
        # Display sample predictions
        print(f"\n📋 Sample future predictions:")
        sample_forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head(6)
        sample_forecast['ds'] = sample_forecast['ds'].dt.strftime('%Y-%m-%d')
        for _, row in sample_forecast.iterrows():
            print(f"   {row['ds']}: {row['yhat']:.3f} (±{(row['yhat_upper']-row['yhat_lower'])/2:.3f})")
            
    except Exception as e:
        print(f"❌ Prediction failed: {e}")
        print(f"   Future dataframe columns: {list(future_with_weather.columns)}")
        print(f"   Model was trained with regressors: {available_regressors + ['region_numeric', 'is_northern_region']}")
        import traceback
        traceback.print_exc()
        forecast = None
        historical_forecast = None
        
else:
    print("❌ Cannot generate predictions - model training failed")
    forecast = None
    historical_forecast = None

📅 Creating predictions from current time: 2025-10-05 10:10:39
   Prediction period: 3 months ahead
   Prediction dates: 24 dates from 2025-01-01 to 2026-01-04
🌡️  Getting weather data for prediction period...
   Prediction spans past and future, combining historical + forecast...
   Fetching forecast from https://api.open-meteo.com/v1/forecast
   Coordinates: (39.0192, -76.8799)
   Date range: 2025-10-06 to 2025-10-20 (limited to 15 days)
   ✅ Retrieved forecast for 15 days
   ⚠️  77 days beyond forecast limit, will use climatology for remainder
🔮 Generating predictions with model...
   Using region_numeric: 0
   Using is_northern_region: 0
   Available regressors: ['ds', 'date', 'temperature_mean', 'humidity', 'cloudcover_mean', 'wind_speed_mean', 'clarity_index', 'vapor_pressure_deficit', 'growing_degree_days', 'precipitation', 'region_numeric', 'is_northern_region', 'is_growing_season']
✅ Predictions generated for 24 future dates
📊 Predicted NDVI range: 0.167 to 0.751

📋 Sample futu

## Blooming Date Prediction

Calculate the predicted blooming start date based on NDVI time series analysis.

In [36]:
def calculate_blooming_start_date(forecast_data, method='ndvi_threshold'):
    """
    Calculate blooming start date based on NDVI predictions
    
    Parameters:
    forecast_data (pd.DataFrame): Prophet forecast with 'ds' and 'yhat' columns
    method (str): Method for calculating blooming start
                  - 'ndvi_threshold': When NDVI reaches threshold value
                  - 'ndvi_acceleration': When NDVI growth accelerates
                  - 'seasonal_pattern': Based on seasonal growth patterns
    
    Returns:
    dict: Blooming prediction results
    """
    if forecast_data is None or len(forecast_data) == 0:
        return {'blooming_start_date': None, 'method': method, 'confidence': 0}
    
    # Create a copy and ensure proper date handling
    df = forecast_data.copy()
    df['ds'] = pd.to_datetime(df['ds'])
    df = df.sort_values('ds')
    
    blooming_results = {}
    
    if method == 'ndvi_threshold':
        # Method 1: Blooming starts when NDVI reaches 0.6-0.7 (typical vegetation vigor for blooming)
        blooming_threshold = 0.65  # Typical NDVI value at blooming onset
        
        # Find first date when NDVI exceeds threshold (during spring growth)
        spring_mask = (df['ds'].dt.month >= 3) & (df['ds'].dt.month <= 6)  # March to June
        spring_data = df[spring_mask]
        
        if len(spring_data) > 0:
            blooming_candidates = spring_data[spring_data['yhat'] >= blooming_threshold]
            if len(blooming_candidates) > 0:
                blooming_date = blooming_candidates['ds'].iloc[0]
                confidence = min(0.9, 0.5 + (blooming_candidates['yhat'].iloc[0] - blooming_threshold) * 2)
                blooming_results = {
                    'blooming_start_date': blooming_date,
                    'method': 'NDVI Threshold (≥0.65)',
                    'confidence': confidence,
                    'ndvi_at_blooming': blooming_candidates['yhat'].iloc[0],
                    'threshold_used': blooming_threshold
                }
    
    elif method == 'ndvi_acceleration':
        # Method 2: Blooming starts when NDVI growth rate accelerates (second derivative > 0)
        if len(df) >= 3:
            # Calculate first and second derivatives
            df['ndvi_growth'] = df['yhat'].diff()
            df['ndvi_acceleration'] = df['ndvi_growth'].diff()
            
            # Find peak acceleration in spring
            spring_mask = (df['ds'].dt.month >= 3) & (df['ds'].dt.month <= 6)
            spring_data = df[spring_mask]
            
            if len(spring_data) > 0:
                # Find when acceleration is highest (rapid NDVI increase)
                max_accel_idx = spring_data['ndvi_acceleration'].idxmax()
                if not pd.isna(max_accel_idx):
                    blooming_date = df.loc[max_accel_idx, 'ds']
                    confidence = min(0.85, 0.6 + abs(df.loc[max_accel_idx, 'ndvi_acceleration']) * 10)
                    blooming_results = {
                        'blooming_start_date': blooming_date,
                        'method': 'NDVI Acceleration Peak',
                        'confidence': confidence,
                        'ndvi_at_blooming': df.loc[max_accel_idx, 'yhat'],
                        'acceleration_value': df.loc[max_accel_idx, 'ndvi_acceleration']
                    }
    
    elif method == 'seasonal_pattern':
        # Method 3: Based on typical seasonal patterns (day of year analysis)
        # Blooming typically occurs between day 90-150 (late March to late May)
        df['day_of_year'] = df['ds'].dt.dayofyear
        blooming_window = df[(df['day_of_year'] >= 90) & (df['day_of_year'] <= 150)].copy()
        
        if len(blooming_window) > 0:
            # Find the steepest NDVI increase in blooming window
            blooming_window['ndvi_change'] = blooming_window['yhat'].diff()
            max_change_idx = blooming_window['ndvi_change'].idxmax()
            
            if not pd.isna(max_change_idx):
                blooming_date = blooming_window.loc[max_change_idx, 'ds']
                confidence = min(0.8, 0.4 + abs(blooming_window.loc[max_change_idx, 'ndvi_change']) * 5)
                blooming_results = {
                    'blooming_start_date': blooming_date,
                    'method': 'Seasonal Pattern Analysis',
                    'confidence': confidence,
                    'ndvi_at_blooming': blooming_window.loc[max_change_idx, 'yhat'],
                    'day_of_year': blooming_window.loc[max_change_idx, 'day_of_year']
                }
    
    # Default return if no method worked
    if not blooming_results:
        blooming_results = {
            'blooming_start_date': None,
            'method': method,
            'confidence': 0,
            'error': 'Unable to determine blooming date with selected method'
        }
    
    return blooming_results

# Calculate blooming dates using multiple methods if forecast is available
if 'forecast' in globals() and forecast is not None and len(forecast) > 0:
    print("🌸 Calculating blooming start dates...")
    
    # Calculate using all three methods
    methods = ['ndvi_threshold', 'ndvi_acceleration', 'seasonal_pattern']
    blooming_predictions = {}
    
    for method in methods:
        result = calculate_blooming_start_date(forecast, method)
        blooming_predictions[method] = result
        
        if result['blooming_start_date'] is not None:
            print(f"   📅 {result['method']}: {result['blooming_start_date'].strftime('%Y-%m-%d')} "
                  f"(Confidence: {result['confidence']:.1%})")
            if 'ndvi_at_blooming' in result:
                print(f"      NDVI at blooming: {result['ndvi_at_blooming']:.3f}")
        else:
            print(f"   ❌ {result['method']}: Could not determine blooming date")
    
    # Select best prediction (highest confidence)
    best_method = max(blooming_predictions.keys(), 
                     key=lambda x: blooming_predictions[x]['confidence'])
    best_prediction = blooming_predictions[best_method]
    
    if best_prediction['blooming_start_date'] is not None:
        print(f"\n🏆 Best blooming prediction:")
        print(f"   Date: {best_prediction['blooming_start_date'].strftime('%Y-%m-%d')}")
        print(f"   Method: {best_prediction['method']}")
        print(f"   Confidence: {best_prediction['confidence']:.1%}")
        print(f"   NDVI at blooming: {best_prediction.get('ndvi_at_blooming', 'N/A')}")
        
        # Calculate days until blooming
        if best_prediction['blooming_start_date'] > pd.Timestamp.now():
            days_until = (best_prediction['blooming_start_date'] - pd.Timestamp.now()).days
            print(f"   Days until blooming: {days_until}")
        
        # Store for future use
        blooming_start_date = best_prediction['blooming_start_date']
        blooming_confidence = best_prediction['confidence']
        blooming_method = best_prediction['method']
        
    else:
        print("\n⚠️  Unable to predict blooming start date with current NDVI data")
        blooming_start_date = None
        blooming_confidence = 0
        blooming_method = None
        
else:
    print("🌸 No forecast data available for blooming prediction")
    print("   Please run the NDVI prediction model first")
    blooming_start_date = None
    bloomingconfidence = 0
    blooming_method = None

🌸 Calculating blooming start dates...
   📅 NDVI Threshold (≥0.65): 2025-05-09 (Confidence: 50.9%)
      NDVI at blooming: 0.655
   📅 NDVI Acceleration Peak: 2025-03-06 (Confidence: 85.0%)
      NDVI at blooming: 0.341
   📅 Seasonal Pattern Analysis: 2025-04-23 (Confidence: 80.0%)
      NDVI at blooming: 0.595

🏆 Best blooming prediction:
   Date: 2025-03-06
   Method: NDVI Acceleration Peak
   Confidence: 85.0%
   NDVI at blooming: 0.3410231601123148


In [37]:
# Create a visualization of blooming predictions
if 'forecast' in globals() and forecast is not None and blooming_start_date is not None:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    print("📊 Creating blooming prediction visualization...")
    
    # Prepare data for plotting
    plot_df = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
    plot_df['ds'] = pd.to_datetime(plot_df['ds'])
    
    # Create the figure
    fig_blooming = go.Figure()
    
    # Add NDVI prediction line
    fig_blooming.add_trace(go.Scatter(
        x=plot_df['ds'],
        y=plot_df['yhat'],
        mode='lines',
        name='NDVI Prediction',
        line=dict(color='green', width=3)
    ))
    
    # Add uncertainty bands
    fig_blooming.add_trace(go.Scatter(
        x=plot_df['ds'].tolist() + plot_df['ds'].tolist()[::-1],
        y=plot_df['yhat_upper'].tolist() + plot_df['yhat_lower'].tolist()[::-1],
        fill='tonexty',
        fillcolor='rgba(0,128,0,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        name='Prediction Uncertainty',
        showlegend=False
    ))
    
    # Add blooming date marker
    blooming_ndvi = blooming_predictions[best_method]['ndvi_at_blooming']
    fig_blooming.add_trace(go.Scatter(
        x=[blooming_start_date],
        y=[blooming_ndvi],
        mode='markers+text',
        marker=dict(
            color='red',
            size=15,
            symbol='star',
            line=dict(color='darkred', width=2)
        ),
        text=[f'🌸 Blooming Start<br>{blooming_start_date.strftime("%Y-%m-%d")}'],
        textposition='top center',
        textfont=dict(size=12, color='darkred'),
        name=f'Blooming Start ({blooming_method})'
    ))
    
    # Add NDVI threshold line if applicable
    if best_method == 'ndvi_threshold':
        threshold = blooming_predictions[best_method]['threshold_used']
        fig_blooming.add_hline(
            y=threshold, 
            line_dash="dash", 
            line_color="red",
            annotation_text=f"Blooming Threshold (NDVI ≥ {threshold})",
            annotation_position="bottom right"
        )
    
    # Update layout
    fig_blooming.update_layout(
        title=dict(
            text=f'NDVI Predictions with Blooming Start Date<br>'
                 f'<sub>Predicted blooming: {blooming_start_date.strftime("%B %d, %Y")} '
                 f'({blooming_confidence:.0%} confidence)</sub>',
            font=dict(size=16)
        ),
        xaxis_title='Date',
        yaxis_title='NDVI',
        hovermode='x unified',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        height=500,
        showlegend=True
    )
    
    # Show the plot
    fig_blooming.show()
    
    # Print summary statistics
    print(f"\n📈 Blooming Prediction Summary:")
    print(f"   🗓️  Predicted blooming start: {blooming_start_date.strftime('%B %d, %Y')}")
    print(f"   🎯 Method used: {blooming_method}")
    print(f"   📊 Confidence level: {blooming_confidence:.1%}")
    print(f"   🌱 NDVI at blooming: {blooming_ndvi:.3f}")
    
    if blooming_start_date > pd.Timestamp.now():
        days_until = (blooming_start_date - pd.Timestamp.now()).days
        print(f"   ⏳ Days until blooming: {days_until} days")
        
        # Calculate weeks until blooming
        weeks_until = days_until // 7
        remaining_days = days_until % 7
        if weeks_until > 0:
            print(f"   📅 Time until blooming: {weeks_until} weeks and {remaining_days} days")
    else:
        print(f"   ⚠️  Blooming date is in the past or has already occurred")
    
    # Compare all methods
    print(f"\n🔍 Comparison of all methods:")
    for method_key, result in blooming_predictions.items():
        if result['blooming_start_date'] is not None:
            print(f"   • {result['method']}: {result['blooming_start_date'].strftime('%Y-%m-%d')} "
                  f"({result['confidence']:.1%})")
        else:
            print(f"   • {result['method']}: No prediction available")
            
else:
    print("📊 No blooming visualization available - missing forecast data or blooming prediction")

📊 Creating blooming prediction visualization...



📈 Blooming Prediction Summary:
   🗓️  Predicted blooming start: March 06, 2025
   🎯 Method used: NDVI Acceleration Peak
   📊 Confidence level: 85.0%
   🌱 NDVI at blooming: 0.341
   ⚠️  Blooming date is in the past or has already occurred

🔍 Comparison of all methods:
   • NDVI Threshold (≥0.65): 2025-05-09 (50.9%)
   • NDVI Acceleration Peak: 2025-03-06 (85.0%)
   • Seasonal Pattern Analysis: 2025-04-23 (80.0%)


## Model Evaluation and Cross-Validation

Evaluate the model performance using validation data and cross-validation techniques.

In [38]:
if model is not None and not val_df.empty:
    print("📊 Evaluating model performance...")
    
    # Add growing season indicator to validation data
    val_df = add_growing_season_indicator(val_df)
    
    # Predict on validation data
    val_forecast = model.predict(val_df)
    
    # Calculate evaluation metrics
    actual = val_df['y'].values
    predicted = val_forecast['yhat'].values
    
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    
    print(f"✅ Validation metrics:")
    print(f"   Mean Absolute Error (MAE): {mae:.4f}")
    print(f"   Root Mean Square Error (RMSE): {rmse:.4f}")
    print(f"   Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
    
    # Cross-validation (if enough data)
    if len(train_df) > 100:  # Only run if sufficient data
        print("\n🔄 Performing cross-validation...")
        try:
            # Create cutoffs for cross-validation (last 6 months of data, with 60-day horizons)
            cutoffs = pd.date_range(
                start=train_df['ds'].max() - pd.DateOffset(months=6),
                end=train_df['ds'].max() - pd.DateOffset(days=60),
                freq='MS'  # Monthly start
            )
            
            if len(cutoffs) > 0:
                cv_results = cross_validation(
                    model, 
                    cutoffs=cutoffs[:3],  # Limit to 3 cutoffs to avoid long runtime
                    horizon='60 days',
                    parallel='processes'
                )
                
                # Performance metrics
                cv_metrics = performance_metrics(cv_results)
                
                print("✅ Cross-validation completed:")
                print(f"   Average MAE: {cv_metrics['mae'].mean():.4f} ± {cv_metrics['mae'].std():.4f}")
                print(f"   Average RMSE: {cv_metrics['rmse'].mean():.4f} ± {cv_metrics['rmse'].std():.4f}")
                print(f"   Average MAPE: {cv_metrics['mape'].mean():.2f}% ± {cv_metrics['mape'].std():.2f}%")
                
            else:
                print("⚠️  Insufficient data for cross-validation")
                cv_results = None
                cv_metrics = None
                
        except Exception as e:
            print(f"⚠️  Cross-validation failed: {e}")
            cv_results = None
            cv_metrics = None
    else:
        print("⚠️  Skipping cross-validation - insufficient training data")
        cv_results = None
        cv_metrics = None
    
    # Feature importance analysis
    print(f"\n🔍 Model component analysis:")
    
    # Get component contributions on validation data
    components = model.predict(val_df)
    
    if 'trend' in components.columns:
        trend_contribution = components['trend'].std()
        print(f"   Trend component variability: {trend_contribution:.4f}")
    
    if 'yearly' in components.columns:
        yearly_contribution = components['yearly'].std()
        print(f"   Yearly seasonality variability: {yearly_contribution:.4f}")
    
    # Regressor contributions
    regressor_importance = {}
    for regressor in available_regressors:
        if regressor in components.columns:
            contrib = components[regressor].std()
            regressor_importance[regressor] = contrib
            print(f"   {regressor} contribution: {contrib:.4f}")
    
    # Store results for later use
    evaluation_results = {
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'regressor_importance': regressor_importance
    }
    
    if cv_metrics is not None:
        evaluation_results['cv_mae'] = cv_metrics['mae'].mean()
        evaluation_results['cv_rmse'] = cv_metrics['rmse'].mean()
        evaluation_results['cv_mape'] = cv_metrics['mape'].mean()
        
else:
    print("⚠️  Skipping evaluation - no model or validation data available")
    evaluation_results = None

📊 Evaluating model performance...
✅ Validation metrics:
   Mean Absolute Error (MAE): 0.1035
   Root Mean Square Error (RMSE): 0.1282
   Mean Absolute Percentage Error (MAPE): 17.38%

🔄 Performing cross-validation...
✅ Cross-validation completed:
   Average MAE: 0.0451 ± 0.0077
   Average RMSE: 0.0546 ± 0.0091
   Average MAPE: 0.28% ± 0.10%

🔍 Model component analysis:
   Trend component variability: 0.0004
   Yearly seasonality variability: 0.0900
   temperature_mean contribution: 0.0019
   humidity contribution: 0.0012
   growing_degree_days contribution: 0.0021
   vapor_pressure_deficit contribution: 0.0057
   precipitation contribution: 0.0023


## Visualize Predictions and Components

Create comprehensive visualizations of the model predictions, components, and validation results.

In [39]:
if model is not None and forecast is not None:
    print("📊 Creating visualizations...")
    
    # 1. Main prediction plot with Plotly
    fig_main = go.Figure()
    
    # Historical data
    fig_main.add_trace(go.Scatter(
        x=train_df['ds'],
        y=train_df['y'],
        mode='markers',
        name='Historical NDVI',
        marker=dict(color='blue', size=6),
        hovertemplate='Date: %{x}<br>NDVI: %{y:.3f}<extra></extra>'
    ))
    
    # Historical predictions
    if historical_forecast is not None:
        fig_main.add_trace(go.Scatter(
            x=historical_forecast['ds'],
            y=historical_forecast['yhat'],
            mode='lines',
            name='Historical Fit',
            line=dict(color='lightblue', width=2),
            hovertemplate='Date: %{x}<br>Predicted: %{y:.3f}<extra></extra>'
        ))
    
    # Future predictions
    fig_main.add_trace(go.Scatter(
        x=forecast['ds'],
        y=forecast['yhat'],
        mode='lines+markers',
        name=f'{CURRENT_YEAR} Predictions',
        line=dict(color='red', width=3),
        marker=dict(size=8),
        hovertemplate='Date: %{x}<br>Predicted NDVI: %{y:.3f}<extra></extra>'
    ))
    
    # Confidence intervals
    fig_main.add_trace(go.Scatter(
        x=forecast['ds'],
        y=forecast['yhat_upper'],
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ))
    
    fig_main.add_trace(go.Scatter(
        x=forecast['ds'],
        y=forecast['yhat_lower'],
        mode='lines',
        fill='tonexty',
        fillcolor='rgba(255,0,0,0.1)',
        line=dict(width=0),
        name='Confidence Interval',
        hovertemplate='Date: %{x}<br>CI: %{y:.3f}<extra></extra>'
    ))
    
    # Validation data if available
    if not val_df.empty:
        fig_main.add_trace(go.Scatter(
            x=val_df['ds'],
            y=val_df['y'],
            mode='markers',
            name='Validation Data',
            marker=dict(color='orange', size=8, symbol='diamond'),
            hovertemplate='Date: %{x}<br>Actual: %{y:.3f}<extra></extra>'
        ))
    
    fig_main.update_layout(
        title=f'NDVI Predictions for {CURRENT_YEAR} - Prophet Model',
        xaxis_title='Date',
        yaxis_title='NDVI',
        height=600,
        hovermode='x unified',
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )
    
    # Add NDVI interpretation lines
    fig_main.add_hline(y=0.2, line_dash="dash", line_color="red", 
                      annotation_text="Stress threshold")
    fig_main.add_hline(y=0.35, line_dash="dash", line_color="orange", 
                      annotation_text="Crop threshold")
    fig_main.add_hline(y=0.6, line_dash="dash", line_color="green", 
                      annotation_text="Healthy canopy")
    
    fig_main.show()
    
    # 2. Seasonal decomposition plot
    print("📊 Creating component analysis...")
    
    if len(forecast) > 0:
        fig_components = make_subplots(
            rows=4, cols=1,
            subplot_titles=['Trend', 'Yearly Seasonality', 'Weather Effects', 'Predictions vs Actual'],
            vertical_spacing=0.08,
            shared_xaxes=True
        )
        
        # Trend
        if 'trend' in forecast.columns:
            fig_components.add_trace(
                go.Scatter(x=forecast['ds'], y=forecast['trend'], 
                          name='Trend', line=dict(color='blue')),
                row=1, col=1
            )
        
        # Seasonality
        if 'yearly' in forecast.columns:
            fig_components.add_trace(
                go.Scatter(x=forecast['ds'], y=forecast['yearly'], 
                          name='Yearly', line=dict(color='green')),
                row=2, col=1
            )
        
        # Weather effects (sum of regressor contributions)
        weather_effect = np.zeros(len(forecast))
        for regressor in available_regressors:
            if regressor in forecast.columns:
                weather_effect += forecast[regressor].values
        
        fig_components.add_trace(
            go.Scatter(x=forecast['ds'], y=weather_effect, 
                      name='Weather Effects', line=dict(color='orange')),
            row=3, col=1
        )
        
        # Predictions vs actual (if validation data available)
        fig_components.add_trace(
            go.Scatter(x=forecast['ds'], y=forecast['yhat'], 
                      name='Predictions', line=dict(color='red')),
            row=4, col=1
        )
        
        if not val_df.empty and len(val_forecast) > 0:
            fig_components.add_trace(
                go.Scatter(x=val_df['ds'], y=val_df['y'], 
                          mode='markers', name='Actual', marker=dict(color='blue')),
                row=4, col=1
            )
        
        fig_components.update_layout(
            height=800,
            title_text="Prophet Model Components Analysis",
            showlegend=True
        )
        
        fig_components.show()
    
    # 3. Weather regressor effects
    if len(available_regressors) > 0:
        print("📊 Creating weather effects visualization...")
        
        fig_weather = make_subplots(
            rows=len(available_regressors), cols=1,
            subplot_titles=[f'{reg.replace("_", " ").title()} Effect' for reg in available_regressors],
            vertical_spacing=0.05,
            shared_xaxes=True
        )
        
        for i, regressor in enumerate(available_regressors):
            if regressor in forecast.columns:
                fig_weather.add_trace(
                    go.Scatter(
                        x=forecast['ds'], 
                        y=forecast[regressor], 
                        name=regressor.replace('_', ' ').title(),
                        line=dict(width=2)
                    ),
                    row=i+1, col=1
                )
        
        fig_weather.update_layout(
            height=150*len(available_regressors),
            title_text="Weather Regressor Effects on NDVI Predictions",
            showlegend=False
        )
        
        fig_weather.show()
    
    # 4. Model performance visualization (if validation data available)
    if evaluation_results is not None:
        print("📊 Creating performance metrics visualization...")
        
        # Create a simple bar chart of metrics
        metrics_fig = go.Figure(data=[
            go.Bar(
                x=['MAE', 'RMSE', 'MAPE (%)'],
                y=[evaluation_results['mae'], evaluation_results['rmse'], evaluation_results['mape']],
                marker_color=['lightblue', 'lightgreen', 'lightcoral']
            )
        ])
        
        metrics_fig.update_layout(
            title='Model Performance Metrics',
            yaxis_title='Error Value',
            height=400
        )
        
        metrics_fig.show()
    
    print("✅ All visualizations created successfully!")
    
else:
    print("❌ Cannot create visualizations - no model or predictions available")

📊 Creating visualizations...


📊 Creating component analysis...


📊 Creating weather effects visualization...


📊 Creating performance metrics visualization...


✅ All visualizations created successfully!


## Export Trained Model and Results

Save the trained model and export prediction results for use in crop monitoring applications.

In [40]:
def estimate_growth_stage(day_of_year, ndvi):
    """Simple growth stage estimation based on day of year and NDVI"""
    if day_of_year < 90 or day_of_year > 300:  # Winter
        return 'Dormant'
    elif day_of_year < 120:  # Early spring
        if ndvi < 0.3:
            return 'Planting/Emergence'
        else:
            return 'Early Growth'
    elif day_of_year < 180:  # Late spring/Early summer
        if ndvi < 0.4:
            return 'Vegetative Growth'
        elif ndvi < 0.6:
            return 'Rapid Growth'
        else:
            return 'Canopy Development'
    elif day_of_year < 240:  # Mid-summer
        if ndvi > 0.6:
            return 'Peak Biomass'
        else:
            return 'Reproduction'
    elif day_of_year < 300:  # Fall
        if ndvi > 0.5:
            return 'Maturation'
        else:
            return 'Senescence'
    else:
        return 'Harvest/Dormancy'

if model is not None:
    print("💾 Saving trained model and results...")
    
    # 1. Save the trained Prophet model
    model_filename = MODEL_OUTPUT_DIR / f"ndvi_prophet_model_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pkl"
    
    try:
        with open(model_filename, 'wb') as f:
            pickle.dump(model, f)
        print(f"✅ Model saved to: {model_filename}")
    except Exception as e:
        print(f"❌ Failed to save model: {e}")
    
    # 2. Save model metadata
    metadata = {
        'model_type': 'Prophet',
        'training_period': {
            'start': train_df['ds'].min().isoformat(),
            'end': train_df['ds'].max().isoformat(),
            'n_observations': len(train_df)
        },
        'regressors': available_regressors,
        'target_variable': 'NDVI',
        'prediction_year': CURRENT_YEAR,
        'created_at': datetime.now().isoformat(),
        'model_parameters': {
            'yearly_seasonality': True,
            'changepoint_prior_scale': 0.05,
            'seasonality_prior_scale': 10.0,
            'n_changepoints': 25
        }
    }
    
    if evaluation_results:
        metadata['performance'] = evaluation_results
    
    metadata_filename = MODEL_OUTPUT_DIR / f"model_metadata_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    try:
        with open(metadata_filename, 'w') as f:
            json.dump(metadata, f, indent=2, default=str)
        print(f"✅ Metadata saved to: {metadata_filename}")
    except Exception as e:
        print(f"❌ Failed to save metadata: {e}")
    
    # 3. Export predictions to CSV
    if forecast is not None:
        predictions_df = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
        predictions_df['ds'] = predictions_df['ds'].dt.strftime('%Y-%m-%d')
        predictions_df.columns = ['date', 'predicted_ndvi', 'lower_bound', 'upper_bound']
        
        # Add weather variables
        for regressor in available_regressors:
            if regressor in forecast.columns:
                predictions_df[regressor] = forecast[regressor]
        
        # Add crop status interpretation
        predictions_df['crop_status'] = predictions_df['predicted_ndvi'].apply(
            lambda x: 'Stressed' if x < 0.2 else
                     'Emerging' if x < 0.35 else
                     'Developing' if x < 0.5 else
                     'Healthy' if x < 0.7 else
                     'Peak'
        )
        
        # Add growth stage estimates (simplified)
        predictions_df['estimated_growth_stage'] = predictions_df.apply(
            lambda row: estimate_growth_stage(
                pd.to_datetime(row['date']).timetuple().tm_yday, 
                row['predicted_ndvi']
            ), axis=1
        )
        
        predictions_filename = MODEL_OUTPUT_DIR / f"ndvi_predictions_{CURRENT_YEAR}.csv"
        
        try:
            predictions_df.to_csv(predictions_filename, index=False)
            print(f"✅ Predictions exported to: {predictions_filename}")
            print(f"   Contains {len(predictions_df)} predictions for {CURRENT_YEAR}")
        except Exception as e:
            print(f"❌ Failed to export predictions: {e}")
    
    # 4. Create summary report
    summary_report = f"""
# NDVI Prophet Model Summary Report
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Model Configuration
- Target Variable: NDVI
- Training Period: {train_df['ds'].min().date()} to {train_df['ds'].max().date()}
- Training Observations: {len(train_df)}
- Prediction Year: {CURRENT_YEAR}
- Weather Regressors: {', '.join(available_regressors) if available_regressors else 'None'}

## Model Performance
"""
    
    if evaluation_results:
        summary_report += f"""- Mean Absolute Error (MAE): {evaluation_results['mae']:.4f}
- Root Mean Square Error (RMSE): {evaluation_results['rmse']:.4f}
- Mean Absolute Percentage Error (MAPE): {evaluation_results['mape']:.2f}%
"""
        
        if 'cv_mae' in evaluation_results:
            summary_report += f"""
## Cross-Validation Results
- CV MAE: {evaluation_results['cv_mae']:.4f}
- CV RMSE: {evaluation_results['cv_rmse']:.4f}
- CV MAPE: {evaluation_results['cv_mape']:.2f}%
"""
    
    if forecast is not None:
        summary_report += f"""
## Predictions Summary
- Prediction Dates: {len(predictions_df)}
- NDVI Range: {forecast['yhat'].min():.3f} to {forecast['yhat'].max():.3f}
- Peak NDVI Expected: {forecast.loc[forecast['yhat'].idxmax(), 'ds'].strftime('%Y-%m-%d')}
- Minimum NDVI Expected: {forecast.loc[forecast['yhat'].idxmin(), 'ds'].strftime('%Y-%m-%d')}

## Growing Season Analysis
- Growing season months (Apr-Oct): {len(forecast[(forecast['ds'].dt.month >= 4) & (forecast['ds'].dt.month <= 10)])} predictions
- Expected peak canopy period: {forecast[forecast['yhat'] > 0.6]['ds'].min().strftime('%Y-%m-%d') if len(forecast[forecast['yhat'] > 0.6]) > 0 else 'Not expected'}

## Crop Monitoring Insights
"""
        
        # Seasonal insights
        monthly_means = forecast.groupby(forecast['ds'].dt.month)['yhat'].mean()
        peak_month = monthly_means.idxmax()
        month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                      'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        
        summary_report += f"""- Predicted peak growth month: {month_names[peak_month-1]} (NDVI: {monthly_means[peak_month]:.3f})
- Early season average (Mar-May): {monthly_means[[3,4,5]].mean():.3f}
- Late season average (Sep-Nov): {monthly_means[[9,10,11]].mean():.3f}

## Files Generated
- Model: {model_filename.name}
- Metadata: {metadata_filename.name}
- Predictions: {predictions_filename.name}
- Summary: model_summary_{datetime.now().strftime('%Y%m%d')}.md
"""
    
    summary_filename = MODEL_OUTPUT_DIR / f"model_summary_{datetime.now().strftime('%Y%m%d')}.md"
    
    try:
        with open(summary_filename, 'w') as f:
            f.write(summary_report)
        print(f"✅ Summary report saved to: {summary_filename}")
    except Exception as e:
        print(f"❌ Failed to save summary: {e}")
    
    print(f"\n🎉 Model export completed! Check the '{MODEL_OUTPUT_DIR}' directory for all files.")

else:
    print("❌ Cannot export - no trained model available")

print("📋 Growth stage estimation function defined")

💾 Saving trained model and results...
✅ Model saved to: models\ndvi_prophet_model_20251005_101111.pkl
✅ Metadata saved to: models\model_metadata_20251005_101111.json
✅ Predictions exported to: models\ndvi_predictions_2025.csv
   Contains 24 predictions for 2025
✅ Summary report saved to: models\model_summary_20251005.md

🎉 Model export completed! Check the 'models' directory for all files.
📋 Growth stage estimation function defined


## Summary and Next Steps

This notebook has demonstrated how to train a Prophet model for NDVI prediction using cached data from the HLS Crop Monitor project.

### Key Accomplishments:
1. ✅ **Data Loading**: Successfully loaded cached NDVI and weather data
2. ✅ **Feature Engineering**: Created weather-based regressors and seasonal features  
3. ✅ **Model Training**: Trained Prophet model with external regressors
4. ✅ **Prediction Generation**: Created NDVI forecasts for the current year
5. ✅ **Model Evaluation**: Assessed performance using validation metrics
6. ✅ **Visualization**: Generated comprehensive plots and analysis
7. ✅ **Export**: Saved model, predictions, and summary reports

### Usage in Production:
- The saved model can be loaded and used to generate updated predictions
- Predictions include confidence intervals for uncertainty quantification
- Weather regressors enable more accurate forecasting
- Growth stage estimates support agricultural decision making

### Potential Improvements:
- **More Weather Variables**: Add soil moisture, solar radiation, wind speed
- **Spatial Modeling**: Extend to multiple field locations
- **Crop-Specific Models**: Train separate models for different crop types
- **Real-time Updates**: Implement automated model retraining
- **Anomaly Detection**: Add alerts for unusual NDVI patterns

### Integration with HLS Crop Monitor:
The predictions from this model can be integrated back into the main Streamlit application to provide:
- Forward-looking crop health forecasts
- Early warning systems for stress conditions
- Seasonal planning support for farmers
- Yield estimation improvements

## Export Model for API Integration

Save the trained Prophet model and metadata for use in the API.

In [41]:
# Export the trained model for API integration
if 'model' in globals() and model is not None:
    import pickle
    
    # Ensure model output directory exists
    MODEL_OUTPUT_DIR.mkdir(exist_ok=True)
    
    print("📦 Exporting trained model for API integration...")
    
    # Save the Prophet model
    model_filename = MODEL_OUTPUT_DIR / "ndvi_prophet_model.pkl"
    
    try:
        with open(model_filename, 'wb') as f:
            pickle.dump(model, f)
        print(f"✅ Model saved to: {model_filename}")
        
        # Save model metadata
        metadata = {
            'model_type': 'Prophet',
            'training_date': datetime.now().isoformat(),
            'regions_trained': list(all_regions),
            'regressors': available_regressors + ['region_numeric', 'is_northern_region', 'is_growing_season'],
            'training_data_shape': list(modeling_df.shape) if 'modeling_df' in globals() else None,
            'cv_metrics': {
                'mae': float(mae) if 'mae' in globals() else None,
                'rmse': float(rmse) if 'rmse' in globals() else None,
                'mape': float(mape) if 'mape' in globals() else None
            } if any(var in globals() for var in ['mae', 'rmse', 'mape']) else None,
            'blooming_prediction_available': True,
            'weather_forecast_integration': True
        }
        
        metadata_filename = MODEL_OUTPUT_DIR / "model_metadata.json"
        with open(metadata_filename, 'w') as f:
            import json
            json.dump(metadata, f, indent=2)
        print(f"✅ Metadata saved to: {metadata_filename}")
        
        # Save example predictions for verification
        if 'forecast' in globals() and forecast is not None:
            predictions_sample = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head(10)
            predictions_filename = MODEL_OUTPUT_DIR / "sample_predictions.csv"
            predictions_sample.to_csv(predictions_filename, index=False)
            print(f"✅ Sample predictions saved to: {predictions_filename}")
        
        # Save blooming prediction results if available
        if 'blooming_start_date' in globals() and blooming_start_date is not None:
            blooming_info = {
                'blooming_start_date': blooming_start_date.isoformat(),
                'blooming_confidence': float(blooming_confidence),
                'blooming_method': blooming_method,
                'ndvi_at_blooming': float(blooming_ndvi) if 'blooming_ndvi' in globals() else None
            }
            
            blooming_filename = MODEL_OUTPUT_DIR / "blooming_prediction_sample.json"
            with open(blooming_filename, 'w') as f:
                # Convert all numeric types to Python native types for JSON serialization
                def convert_for_json(obj):
                    if isinstance(obj, pd.Timestamp):
                        return obj.isoformat()
                    elif isinstance(obj, (np.integer, np.int32, np.int64)):
                        return int(obj)
                    elif isinstance(obj, (np.floating, np.float32, np.float64)):
                        return float(obj)
                    elif isinstance(obj, dict):
                        return {k: convert_for_json(v) for k, v in obj.items()}
                    elif isinstance(obj, list):
                        return [convert_for_json(item) for item in obj]
                    return obj
                
                json.dump(convert_for_json(blooming_info), f, indent=2)
            print(f"✅ Blooming predictions saved to: {blooming_filename}")
        
        print(f"\n🎉 Model export complete!")
        print(f"   📁 Model files location: {MODEL_OUTPUT_DIR}")
        print(f"   🔧 The API can now use your trained ML model")
        print(f"   🌸 Includes blooming date predictions")
        print(f"   📊 Model performance metrics included")
        
        # Display integration instructions
        print(f"\n📋 API Integration Instructions:")
        print(f"   1. The API will automatically load the model from {model_filename}")
        print(f"   2. ML predictions will replace heuristic NDVI peak calculations")
        print(f"   3. Blooming dates will be included in API responses")
        print(f"   4. The API will fall back to heuristic method if ML fails")
        
    except Exception as e:
        print(f"❌ Error exporting model: {e}")
        import traceback
        traceback.print_exc()
        
else:
    print("⚠️  No trained model available for export")
    print("   Please run the model training cells first")

📦 Exporting trained model for API integration...
✅ Model saved to: models\ndvi_prophet_model.pkl
✅ Metadata saved to: models\model_metadata.json
✅ Sample predictions saved to: models\sample_predictions.csv
✅ Blooming predictions saved to: models\blooming_prediction_sample.json

🎉 Model export complete!
   📁 Model files location: models
   🔧 The API can now use your trained ML model
   🌸 Includes blooming date predictions
   📊 Model performance metrics included

📋 API Integration Instructions:
   1. The API will automatically load the model from models\ndvi_prophet_model.pkl
   2. ML predictions will replace heuristic NDVI peak calculations
   3. Blooming dates will be included in API responses
   4. The API will fall back to heuristic method if ML fails
